#### Import Libraries

In [191]:
import pandas as pd
from goodreads import client
import numpy as np
from bs4 import BeautifulSoup
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import gensim

#### Import Ratings Data

In [2]:
ratings = pd.read_csv('../ratings.csv')

In [4]:
ratings.shape

(5976479, 3)

In [3]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [5]:
book_ids = ratings.book_id.unique()

In [6]:
len(book_ids)

10000

#### Import Book Metadata

In [13]:
book_info = pd.read_csv('../books.csv')

In [14]:
book_info.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [26]:
book_info.columns

Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')

#### Merge in Relevant Metadata

In [28]:
ratings = ratings.merge(right=book_info[['book_id', 'isbn', 'authors', 'original_publication_year', 
                                         'original_title']], on='book_id')

#### Get Book Summaries

In [8]:
gc = client.GoodreadsClient('l2td8Sa3lHfqooxdH5w','i5ReUzTEBrPfoEo1wKCc3VHUeU8KodEzUDwTBP0')

In [151]:
def get_clean_summary(b_id, ratings):
    isbn = ratings.isbn[ratings['book_id'] == b_id].iloc[0]
    if isinstance(isbn, str):
        while len(isbn) < 10:
            isbn = '0' + isbn
        try:
            item = gc.book(isbn=isbn)
        except:
            try:
                isbn = isbn[1:]
                item = gc.book(isbn=isbn)
            except:
                return None
        desc = item.description
        if desc is not None:
            clean = BeautifulSoup(desc, 'lxml').text
            return clean
        else:
            return None
    else:
        return None

In [138]:
count

6183

In [139]:
summaries[-1]

{'book_id': 3218,
 'description': "SOME NIGHTMARES NEVER END.For Janie and Cabel, real life is getting tougher than the dreams. They're just trying to carve out a little (secret) time together, but no such luck.Disturbing things are happening at Fieldridge High, yet nobody's talking. When Janie taps into a classmate's violent nightmares, the case finally breaks open -- but nothing goes as planned. Not even close. Janie's in way over her head, and Cabe's shocking behavior has grave consequences for them both.Worse yet, Janie learns the truth about herself and her ability -- and it's bleak. Seriously, brutally bleak. Not only is her fate as a dream catcher sealed, but what's to come is way darker than she'd feared....From the inside of ISBN 1416953582/9781416953586"}

In [144]:
book_ids[6181]

8422

In [149]:
ratings[ratings['book_id'] == 8422].iloc[0]

user_id                                       7686
book_id                                       8422
rating                                           4
isbn                                     743292511
authors                                  Greg Iles
original_publication_year                     2009
original_title               The Devil's Punchbowl
Name: 4470104, dtype: object

In [150]:
item = gc.book(isbn='0743292511')

TypeError: __str__ returned non-string (type tuple)

In [152]:
# summaries = []
# count = 0
for book_id in book_ids[6181:]:
    count += 1
    if count % 100 == 0:
        print(count)
    new_dict = {'book_id': book_id}
    new_dict['description'] = get_clean_summary(book_id, ratings)
    summaries.append(new_dict)

6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000


In [153]:
summaries_df = pd.DataFrame(summaries)

In [154]:
summaries_df.to_csv('summaries.csv')

In [155]:
summaries_df.shape

(10000, 2)

In [156]:
summaries_df.head()

,book_id,description
0,258,"The international literary sensation, about a ..."
1,4081,"Tom Wolfe, the master social novelist of our t..."
2,260,None
3,9296,As charming performers who skillfully reflect ...
4,2318,The incredible national bestseller that is cha...


#### Merge in Summaries

In [157]:
ratings = ratings.merge(right=summaries_df, on='book_id')

In [158]:
ratings.head()

,user_id,book_id,rating,isbn,authors,original_publication_year,original_title,description
0,1,258,5,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,"The international literary sensation, about a ..."
1,11,258,3,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,"The international literary sensation, about a ..."
2,143,258,4,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,"The international literary sensation, about a ..."
3,242,258,5,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,"The international literary sensation, about a ..."
4,325,258,4,143034901,"Carlos Ruiz Zafón, Lucia Graves",2001.0,La sombra del viento,"The international literary sensation, about a ..."


#### NLP on summaries

In [186]:
valid_summaries = pd.DataFrame(summaries_df[summaries_df.description.notnull()]['description'])

In [194]:
def get_tokens(row):
    tokens = nltk.word_tokenize(row['description'])
    no_punc = [i for i in tokens if i not in list(string.punctuation)]
    return no_punc

In [195]:
valid_summaries['tokens'] = valid_summaries.apply(lambda row: get_tokens(row), axis=1)

In [196]:
valid_summaries.head()

,description,tokens
0,"The international literary sensation, about a ...","[The, international, literary, sensation, abou..."
1,"Tom Wolfe, the master social novelist of our t...","[Tom, Wolfe, the, master, social, novelist, of..."
3,As charming performers who skillfully reflect ...,"[As, charming, performers, who, skillfully, re..."
4,The incredible national bestseller that is cha...,"[The, incredible, national, bestseller, that, ..."
5,A fascinating and absorbing thriller -- perfec...,"[A, fascinating, and, absorbing, thriller, --,..."


#### ngrams

In [197]:
vect = CountVectorizer(stop_words='english', ngram_range=(1,3), min_df=2)

In [199]:
ngrams = vect.fit_transform(valid_summaries.description)

In [200]:
ngrams

<9060x91870 sparse matrix of type '<class 'numpy.int64'>'
	with 806274 stored elements in Compressed Sparse Row format>

#### tf-idf

In [201]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,3), min_df=2)

In [202]:
ngrams_tfidf = tfidf.fit_transform(valid_summaries.description)

In [203]:
ngrams_tfidf

<9060x91870 sparse matrix of type '<class 'numpy.float64'>'
	with 806274 stored elements in Compressed Sparse Row format>

#### word2vec

In [182]:
LabeledSentence = gensim.models.doc2vec.LabeledSentence

In [183]:
def read_texts(docs, tags):
    for idx, doc in enumerate(docs):
        yield LabeledSentence(words=doc, tags=tags)

In [204]:
it = read_texts(valid_summaries['tokens'], valid_summaries.index)

In [205]:
model = gensim.models.Doc2Vec(size=300, window=10, min_count=5, alpha=0.025)
model.build_vocab(it)

In [206]:
model.train(it, total_examples=model.corpus_count, epochs=20)

0

In [219]:
similar_list = model.docvecs.most_similar(0)

In [221]:
with pd.option_context('display.width', 180):
    print(valid_summaries.iloc[0]['description'])
    for item in similar_list:
        print('\n')
        print(valid_summaries.loc[item[0]]['description'])

The international literary sensation, about a boy's quest through the secrets and shadows of postwar Barcelona for a mysterious author whose book has proved as dangerous to own as it is impossible to forget.Barcelona, 1945 - just after the war, a great world city lies in shadow, nursing its wounds, and a boy named Daniel awakes on his eleventh birthday to find that he can no longer remember his mother’s face. To console his only child, Daniel’s widowed father, an antiquarian book dealer, initiates him into the secret of the Cemetery of Forgotten Books, a library tended by Barcelona’s guild of rare-book dealers as a repository for books forgotten by the world, waiting for someone who will care about them again. Daniel’s father coaxes him to choose a volume from the spiraling labyrinth of shelves, one that, it is said, will have a special meaning for him. And Daniel so loves the novel he selects, The Shadow of the Wind by one Julian Carax, that he sets out to find the rest of Carax’s wor

#### GloVe